In [ ]:
import pandas
import datetime
import numpy as np
import os

d = '2022-09-30 00:00:00'
txn_dirPath = '/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/'


txn_converter = {'TXN - Transaction Type': str,
                 'TXN - Transaction Date': str,
                 'TXN - Item ID': str,
                 'TXN - Qty': float,
                 'TXN - Total Cost': float,
                 'TXN - Adjust Type': str}

#RC: add TXN-Sequence Nbr
transaction_col = ['TXN - Sequence Nbr', 'TXN - Transaction Type', 'TXN - Transaction Date',
                   'TXN - Item ID', 'TXN - Qty', 'TXN - Total Cost', 'TXN - Adjust Type']

out_types = ['051', '054,', '030', '031', '012']
positive_types = ['041', '022', '024', '050', '010', '020']
drop_types = ['053', '060']


# return all excel files in the directory as a list
def get_lof(directory):
    lof = []
    for file in os.listdir(directory):
        if file.endswith('.xlsx'):
            lof.append(directory + file)
    return lof


def read_txn_to_df(lof, converter):
    dataframe = pandas.DataFrame()
    for f in lof:
        next_txn = pandas.read_excel(f, converters=converter)
        #RC: changed to concat
        dataframe = pandas.concat([dataframe, next_txn])
        print(f + ' is appended')
    return dataframe


# handle the increase/decrease column
def handle_I_D(txn_df):
    txn_df['TXN - Qty'] = np.where(txn_df['TXN - Adjust Type'] == 'D', 0 - abs(txn_df['TXN - Qty']),
                                   txn_df['TXN - Qty'])
    txn_df['TXN - Qty'] = np.where(txn_df['TXN - Adjust Type'] == 'M', 0 - abs(txn_df['TXN - Qty']),
                                   txn_df['TXN - Qty'])
    txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Adjust Type'] == 'D', 0 - abs(txn_df['TXN - Total Cost']),
                                          txn_df['TXN - Total Cost'])
    txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Adjust Type'] == 'M', 0 - abs(txn_df['TXN - Total Cost']),
                                          txn_df['TXN - Total Cost'])
    
    #RC: removed abs(), changed syntax
    #txn_df['TXN - Qty'] = np.where(txn_df['TXN - Transaction Type'].isin(out_types), 0 - txn_df['TXN - Qty'],
    #                               txn_df['TXN - Qty'])
    #txn_df['TXN - Total Cost'] = np.where(txn_df['TXN - Transaction Type'].isin(out_types), 0 - txn_df['TXN - Total Cost'],
    #                                      txn_df['TXN - Total Cost'])

    # Trust Anni's Code – it has already been validated.

    # We don't want to drop columns.
    #txn_df = txn_df.drop(columns=['TXN - Transaction Type', 'TXN - Adjust Type'])
    return txn_df

# eg: date = '2021-11-01 00:00:00'
# return all txn before 2021-11-01
def read_txn_by_date(txn_df, date):
    txn_df = txn_df.loc[txn_df['TXN - Transaction Date'] < date]
    #txn_df = txn_df.drop(columns=['TXN - Transaction Date'])
    # We don't want to drop the Transaction Date column.
    return txn_df


In [2]:
txn_df = read_txn_to_df(get_lof(txn_dirPath), txn_converter)

/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2004_1Hx.xlsx is appended
/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2004_Q3x.xlsx is appended
/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2004_Q4x.xlsx is appended
/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2005_Q1x.xlsx is appended
/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2005_Q2x.xlsx is appended
/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2005_Q3x.xlsx is appended
/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2005_Q4x.xlsx is appended
/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2006_Q1x.xlsx is appended
/Users/rcheung/Documents/inv_mgmt_report/transaction_rollup/PL_INVENTORY_TRANSACTIONS_2006_Q2x.xlsx is a

In [ ]:
item_wo_txn = pandas.read_excel("wo_ins/wo_txns.xlsx", converters=converter)

In [ ]:
# dataframe = txn_df.copy(deep = True)
#RC: switched order of filter and drop duplicate
# filter dataframe
dataframe = txn_df.loc[txn_df['TXN - Transaction Type'].isin(['010', '020', '012', '022', '024', '030', '031', '041', '050', '051', '054', '053', '060'])]

# drop duplication
dataframe = dataframe.drop_duplicates()

#keep relevant rows
dataframe = dataframe[transaction_col]

#dataframe = read_txn_by_date(dataframe,d)
dataframe = handle_I_D(dataframe)
dataframe

,TXN - Sequence Nbr,TXN - Transaction Type,TXN - Transaction Date,TXN - Item ID,TXN - Qty,TXN - Total Cost,TXN - Adjust Type
0,4794,020,2004-04-02 00:00:00,04567215,2.0,179.78,NaN
1,13692,020,2004-04-02 00:00:00,04260657,10.0,46.11,NaN
2,8907,020,2004-04-02 00:00:00,02005067,50.0,7.03,NaN
3,8704,020,2004-04-02 00:00:00,05180130,2.0,40.44,NaN
4,4899,020,2004-04-02 00:00:00,04587046,3.0,857.55,NaN
...,...,...,...,...,...,...,...
9276,1,041,2022-10-05 00:00:00,03501351,0.0,0.00,NaN
9277,1,041,2022-10-05 00:00:00,02690605,0.0,0.00,NaN
9278,2,041,2022-10-05 00:00:00,02690605,0.0,0.00,NaN
9279,1,041,2022-10-05 00:00:00,02692249,0.0,0.00,NaN


In [5]:
# calculate outs
df_outgoing = dataframe.loc[dataframe['TXN - Transaction Type'].isin(out_types)]
# group by date and item ID fields
df_outgoing = df_outgoing.groupby(['TXN - Item ID', 'TXN - Transaction Date']).agg(
                        {'TXN - Qty': 'sum', 'TXN - Total Cost': 'sum' }).reset_index()

# calculate ins
df_incoming = dataframe.loc[dataframe['TXN - Transaction Type'].isin(positive_types)]
# group by date and item ID fields
df_incoming = df_incoming.groupby(['TXN - Item ID', 'TXN - Transaction Date']).agg(
                        {'TXN - Qty': 'sum', 'TXN - Total Cost': 'sum' }).reset_index()

In [6]:
# combine ingoing and outgoing dfs by item and date
df_byitem = pandas.merge(df_outgoing, df_incoming, on=['TXN - Item ID', 'TXN - Transaction Date'], how='outer')
df_byitem.update(df_byitem[['TXN - Qty_x', 'TXN - Total Cost_x', 'TXN - Qty_y',	'TXN - Total Cost_y']].fillna(0))

rename_dict = {'TXN - Qty_x':'TXN - Outgoing Qty', 'TXN - Total Cost_x':'TXN - Outgoing Total Cost', 'TXN - Qty_y':'TXN - Incoming Qty', 'TXN - Total Cost_y':'TXN - Incoming Total Cost'}

# call rename () method
df_byitem.rename(columns= rename_dict, inplace=True)

df_merged = df_byitem.sort_values(by = ['TXN - Item ID', 'TXN - Transaction Date'], ascending = [True, True], na_position = 'first').reset_index(drop=True)

In [7]:
for i in range(0, len(df_byitem)):
    if i > 0 and df_byitem.loc[i, 'TXN - Item ID'] == df_byitem.loc[i-1, 'TXN - Item ID']:
        df_byitem.loc[i, 'TXN - Overall Qty'] = df_byitem.loc[i-1, 'TXN - Overall Qty'] + df_byitem.loc[i, 'TXN - Incoming Qty'] - df_byitem.loc[i, 'TXN - Outgoing Qty']
        df_byitem.loc[i, 'TXN - Overall Total Cost'] = df_byitem.loc[i-1, 'TXN - Overall Total Cost'] + df_byitem.loc[i, 'TXN - Incoming Total Cost'] - df_byitem.loc[i, 'TXN - Outgoing Total Cost']
    else:
        df_byitem.loc[i, 'TXN - Overall Qty'] = df_byitem.loc[i, 'TXN - Incoming Qty'] - df_byitem.loc[i, 'TXN - Outgoing Qty']
        df_byitem.loc[i, 'TXN - Overall Total Cost'] = df_byitem.loc[i, 'TXN - Incoming Total Cost'] -  df_byitem.loc[i, 'TXN - Outgoing Total Cost']

#df_merged['TXN - Overall Qty'] = df_merged.apply(lambda x: x['TXN - Outgoing Qty'] + x['TXN - Incoming Qty'], axis = 1)
#df_merged['TXN - Overall Total Cost'] = df_merged.apply(lambda x: x['TXN - Outgoing Total Cost'] + x['TXN - Incoming Total Cost'], axis = 1)

In [8]:
df_byitem

,TXN - Item ID,TXN - Transaction Date,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,00118003,2007-02-07 00:00:00,-13.0,-88.49,0.0,0.00,-13.0,-88.49
1,00118003,2007-05-07 00:00:00,-13.0,-88.49,13.0,88.49,-13.0,-88.49
2,00118003,2014-03-25 00:00:00,-5.0,-34.03,0.0,0.00,-18.0,-122.52
3,00118004,2004-09-09 00:00:00,-1.0,-8.79,0.0,0.00,-1.0,-8.79
4,00118004,2005-01-21 00:00:00,-1.0,-8.79,0.0,0.00,-2.0,-17.58
...,...,...,...,...,...,...,...,...
5069472,YAV95M6,2019-04-02 00:00:00,0.0,0.00,-150.0,-150.00,0.0,0.00
5069473,Z53921,2017-09-27 00:00:00,0.0,0.00,173.0,173.00,173.0,173.00
5069474,Z53921,2019-04-02 00:00:00,0.0,0.00,-173.0,-173.00,0.0,0.00
5069475,ZX0201015,2007-09-25 00:00:00,0.0,0.00,51.0,512.55,51.0,512.55


In [9]:
df_bydate = df_merged.groupby(['TXN - Transaction Date']).agg(
                        {'TXN - Outgoing Qty': 'sum', 'TXN - Outgoing Total Cost': 'sum', 'TXN - Incoming Qty': 'sum', 'TXN - Incoming Total Cost': 'sum'}).reset_index()

for i in range(0, len(df_bydate)):
    if i > 0:
        df_bydate.loc[i, 'TXN - Overall Qty'] = df_bydate.loc[i-1, 'TXN - Overall Qty'] + df_bydate.loc[i, 'TXN - Incoming Qty'] + df_bydate.loc[i, 'TXN - Outgoing Qty']
        df_bydate.loc[i, 'TXN - Overall Total Cost'] = df_bydate.loc[i-1, 'TXN - Overall Total Cost'] + df_bydate.loc[i, 'TXN - Incoming Total Cost'] + df_bydate.loc[i, 'TXN - Outgoing Total Cost']
    else:
        df_bydate.loc[i, 'TXN - Overall Qty'] = df_bydate.loc[i, 'TXN - Incoming Qty'] + df_bydate.loc[i, 'TXN - Outgoing Qty']
        df_bydate.loc[i, 'TXN - Overall Total Cost'] = df_bydate.loc[i, 'TXN - Incoming Total Cost'] + df_bydate.loc[i, 'TXN - Outgoing Total Cost']

In [10]:
df_bydate

,TXN - Transaction Date,TXN - Outgoing Qty,TXN - Outgoing Total Cost,TXN - Incoming Qty,TXN - Incoming Total Cost,TXN - Overall Qty,TXN - Overall Total Cost
0,2004-04-02 00:00:00,0.0,0.00,645248.0,3454904.17,645248.00,3.454904e+06
1,2004-04-03 00:00:00,-8521.0,-36669.87,4700490.0,41905102.64,5337217.00,4.532334e+07
2,2004-04-05 00:00:00,-2029.0,-23199.52,666.0,22389.05,5335854.00,4.532253e+07
3,2004-04-06 00:00:00,-16886.0,-94403.35,2232.0,54497.61,5321200.00,4.528262e+07
4,2004-04-07 00:00:00,-39463.0,-89634.75,32288.0,160379.11,5314025.00,4.535337e+07
...,...,...,...,...,...,...,...
6578,2022-10-01 00:00:00,-821.0,-24613.00,0.0,0.00,5214096.67,6.607192e+07
6579,2022-10-02 00:00:00,-1224.0,-28814.91,0.0,0.00,5212872.67,6.604310e+07
6580,2022-10-03 00:00:00,-11934.0,-153810.30,14860.0,172298.13,5215798.67,6.606159e+07
6581,2022-10-04 00:00:00,-9691.0,-193010.00,8718.0,286804.83,5214825.67,6.615539e+07


In [11]:
df_byitem.to_csv('txns_by_item.csv', index = None)
df_bydate.to_csv('txns_by_date.csv', index = None)

In [13]:
dataframe.to_csv('txns_raw.csv', index = None)